In [4]:
import pdfplumber

In [27]:
area_of_law = "Area – Of – Law"

facts_set = ["The facts", 
             "Facts", 
             "The background", 
             "Background", 
             "Factual Background", 
             "Background facts", 
             "Background to the dispute", 
             "background", 
             "Background to", 
             "The underlying facts", 
             "Pertinent background facts"]

issues_set = [
    "The issues",
    "The appeal",
    "Appeal",
    "The issues on appeal",
    "The claim",
    "The present claim",
    "Background to the dispute",
    "The substantive issue",
    "Issue to be determined",
    "The relevant issues"
]

def extract_facts(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text_extracted= ""
        text_extracted2 = ""

        # get the facts header in this pdf and the page number the issues are on
        h = ""
        i = ""
        issues_page_number = 0

        extract_page = False
        set_next_page = False
        for page in pdf.pages:
            text = page
            #Find BOLD text (meaning headers only)
            clean_text = text.filter(lambda obj: obj["object_type"] == "char" and "Bold" in obj["fontname"])
            lst = clean_text.extract_text().split("\n")
            #print(lst)
            for header in lst:
                if set_next_page:
                    extract_page = False
                    set_next_page = False
                if header in facts_set: #Start extracting only when header is for facts
                    extract_page = True
                    h = header
                if header in issues_set: #End extraction when header is for issues
                    set_next_page = True
                    issues_page_number = page.page_number
                    i = header
            if extract_page:
                page_text = text.extract_text()
                text_extracted += page_text + "/n"

        issues_page = pdf.pages[issues_page_number - 1]
        issues_page_text = issues_page.extract_text()

        index_i = issues_page_text.find(i)
        if index_i != -1:
            text_extracted2 = issues_page_text[:index_i]

        # delete the front parts of the page before the facts
        index_h = text_extracted.find(h)
        if index_h != -1:
            text_extracted = text_extracted[index_h:]
        
        return text_extracted + text_extracted2



How the code works: Looks for page with facts-related header, gets text from that page. To account for "spillovers" into the next page, look for page with presumably the next header which is issues-related, and extract the text from that page too. Subsequently, filter only facts-related info.

Limitations: Will not be accurate for headings that are not in the predetermined sets. Perhaps I should modify such that it detects the *next* heading instead. Also, if there are exact same terms in the text as the predetermined sets, it may cut off the text prematurely.

In [28]:
file = "/Users/matthewlyj/Documents/GitHub/case-outcome-predictor/raw-cases/2000_SGCA_4.pdf"
print(extract_facts(file))

The facts
Dauphin is a private limited company incorporated in Singapore and is in the business of, inter alia,
shipbuilding. HRH is a corporation organised and established under the laws of the United Arab
Emirates (`UAE`).
By a contract dated 2 August 1998 (`the contract`) HRH engaged Dauphin to build a 55-metre twin-
screw luxury motor yacht at the price of US$5,850,000. The contract provides that HRH shall make
payment for the construction of the yacht by instalments. For the purpose of this action only the
terms governing the first and second instalment payments are relevant and they are as follows:
First instalment
The sum of US$877,500, equivalent to 15% of the contract sum, shall be paid
within five days of the effective date of the contract by telegraphic transfer of
the amount into Dauphin`s bank account.
Second instalment
The sum of US$1,462,500, equivalent to 25% of the contract sum shall be paid
upon the keel being laid. This instalment shall be paid by telegraphic transfe